In [2]:
from ArticlesRep import MeanSimilarityoneindustry,MeanSimilaritytwoindustry,MakeMatrixTSEN #common function

import pandas as pd

import numpy as np

%matplotlib inline

import pickle

import visdom

from tqdm import tnrange, tqdm_notebook
from tqdm.autonotebook import tqdm
tqdm.pandas()

from time import time,sleep
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
# import visdom
# vis=visdom.Visdom()
# env="TagBased"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

import re
import sys
from pprint import pprint
#coding:utf-8

#solved chinese display in matplotlib
from pylab import mpl
mpl.rcParams["font.family"] = 'DFKai-sb' 
mpl.rcParams['axes.unicode_minus'] = False

list_industry=["水泥","食品飲料","石化","紡織","電機機械","電器電纜","化學工業",
               "建材居家用品","造紙","鋼鐵金屬","車輛相關","科技相關","營建地產","運輸","觀光休閒娛樂",
               "金融相關","百貨通路","公用事業","控股","生技醫療保健","農林漁牧","航天軍工","能源","傳播出版","綜合",
               "傳產其他","其他","金屬礦採選",
              ]
element="氫氦鋰鈹硼碳氮氧氟氖鈉鎂鋁矽磷硫氯氬鉀鈣鈧鈦釩鉻錳鐵鈷鎳銅鋅鎵鍺砷硒溴氪銣鍶銀鎘銦錫銻碲碘氙銫鋇鉑金汞鉈鉛鉍釙氡鍅鐳"

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Outline
<li>Text to Vector version1
<li>Text to Vector version2
<li>Text to Vecotr multi-industry

## Text to Vector version1
<li> Load DataSet_vip_single_industry
<li> Vectorization Binary one hot of bag word (first 20,000 popular word)
<li> Save 
<li> Load
<li> Sparcisity

### Load DataSet_vip_single_industry

In [2]:
with open("D:3.AutoencoderForArticle/DataSet_vip_single_industry","rb") as f:
    DataSet=pickle.load(f)

In [3]:
DataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154884 entries, 0 to 154883
Data columns (total 3 columns):
guid                           154884 non-null object
title_token_plus_body_token    154884 non-null object
indusrty_tags                  154884 non-null object
dtypes: object(3)
memory usage: 3.5+ MB


In [31]:
DataSet[:5]

,guid,title_token_plus_body_token,indusrty_tags
0,4dd5651a-8d9d-4c38-b658-696242109941,Fund 消息 資金 連 周 回流 收債 後市 看 樂觀 美國 景氣 維持 暢旺 加上 市場...,{鋼鐵金屬}
1,8b6822c0-e646-4592-a783-0aaf54e9aa6f,安心 今年 高度 挑戰 一年 自助 點餐 機助 效率 提升 餐飲業 人事 成本 持續 攀升 ...,{觀光休閒娛樂}
2,2a05600a-75ff-4f0e-be07-97c4362e88d6,信義 房市 北溫 南冷 大雨 民俗 澆熄 熱度 信義 房屋 統計 全台 都會區 門市 交易 ...,{營建地產}
3,494d18ba-75d1-4276-8fb3-cccec13eebaa,陸 鋼鐵 PMI 降至 後市 鋼價料 高檔 支撐 西 本 新幹線 報導 物聯 鋼鐵 物流 專...,{鋼鐵金屬}
4,d219315e-5b1d-41ba-b89a-7381f39c133a,巴菲特 蘋果 別 投資 特斯拉 馬斯克 太常 推文 利生 產 特斯拉 Tesla Inc 執...,{車輛相關}


In [5]:
_=DataSet["title_token_plus_body_token"]
_=_.value_counts()
_.reset_index().rename(columns={"index":"title_token_plus_body_token","title_token_plus_body_token":"counts"})

,title_token_plus_body_token,counts
0,美股 指數期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJ IA INDEX a...,3074
1,股東會 資訊 好消息 台 總 報給 您 知 總 股份有限公司 徵求 股東會 委託書 提供 紀...,47
2,理財 短波 線上 開戶 首選 合庫 讓 您 證券 開戶 更 便利 合庫 證券 去 正式 啟動...,10
3,嘉實 學院 課程 招生 投資 美股 研習班 嘉實 學院 首次 開辦 美股 課程 特別 邀請 ...,7
4,理財 短波 線上 開戶 首選 合庫 讓 您 證券 開戶 更 便利 合庫 證券 於今 正式 啟...,7
5,理財 短波 線上 開戶 首選 合庫 讓 您 證券 開戶 更 便利 合庫 證券 於今 正式 啟...,5
6,日本 銀行 放款 餘額 年減 連 個 下滑 根據 日本央行 BOJ 公佈 日本 全 國 銀行...,4
7,日本經濟 銀行 放款 額 年增 連 個 增長 根據 日本央行 BOJ 公佈 日本 全 國 銀...,4
8,日本經濟 銀行 放款 餘額 年減 連 個 下滑 根據 日本央行 BOJ 公佈 日本 全 國 ...,3
9,日本央行 決議 短期 利率 誘導 目標 維持 不變 日本央行 BOJ 分 當地 時間 發佈 ...,3


### Vectorization

In [52]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [7]:
texts = DataSet["title_token_plus_body_token"]
tokenizer = Tokenizer(num_words=20000,)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# word_index

Found 272511 unique tokens.


In [8]:
_=pd.Series(tokenizer.word_counts)
_.sort_values(ascending=False,inplace=True)

In [9]:
_[:20000][[0,1,2,3,4,5,-6,-5,-4,-3,-2,-1]]

營收       252141
億元       209036
今年       202674
表示       187343
市場       180937
公司       158782
短空           99
房仲業          99
支持率          99
天膠           99
效用           99
led基板        99
dtype: int64

In [10]:
_=pd.Series(tokenizer.word_docs)
_.sort_values(ascending=False,inplace=True)
_[:20000][[0,1,2,3,4,5,-6,-5,-4,-3,-2,-1]]

表示      95150
而       77839
市場      75096
今年      71997
指出      71724
公司      67069
愈加         70
施打         70
有信心的       70
可延長        70
呈報         70
數位經濟       70
dtype: int64

In [11]:
one_hot_results = np.asarray(tokenizer.texts_to_matrix(texts, mode='binary'),dtype="int8")
one_hot_results

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]], dtype=int8)

### Save one_hot_results 

In [13]:
np.save("D:3.AutoencoderForArticle/BOW_binary_v01_int8.npy",one_hot_results)

### Load one_hot_results 

In [14]:
# BOW_binary=np.load("D:3.AutoencoderForArticle/BOW_binary_v01.npy")

In [7]:
#sparcity
sparcity=np.mean(BOW_binary)
print("{}%".format(np.round(sparcity,5)*100))

0.616%


In [8]:
np.sum(BOW_binary[69520])

203

## Text to Vector version2
<li> Load DataSet_vip_single_industry
<li> Build Vocabulary(commenest 3700 words per industry)
<li> Use Vocanbulry to vectrize text  
<li> Sparcisity
<li> Save

#### Load DataSet_vip_single_industry

In [79]:
with open("D:3.AutoencoderForArticle/DataSet_vip_single_industry","rb") as f:
    DataSet=pickle.load(f)

In [80]:
from keras.preprocessing.text import Tokenizer

#### Build Vocabulary

In [81]:
dictindustryvocabulary=dict()
n=3700
for idu in  tqdm_notebook(list_industry):
    text=DataSet[DataSet["indusrty_tags"]=={idu}]["title_token_plus_body_token"].values
    tokenizer=Tokenizer(num_words=n)
    tokenizer.fit_on_texts(text)
    
    df=pd.Series(tokenizer.word_index)
    wordset=df.sort_values(ascending=True).index[:n]
    
    dictindustryvocabulary[idu]=wordset

In [82]:
vocabulary=set()
allhave=set()
for i,k in enumerate(dictindustryvocabulary):
#     print(len(dictindustryvocabulary[k]))
    vocabulary=vocabulary|set(dictindustryvocabulary[k])
    if i==0:allhave=allhave|set(dictindustryvocabulary[k])
    allhave=allhave&set(dictindustryvocabulary[k])

In [83]:
len(vocabulary)

20506

In [84]:
word_industrys={i:0 for i in vocabulary}
for k in dictindustryvocabulary:
    for word in dictindustryvocabulary[k]:
        word_industrys[word]+=1

In [85]:
df=pd.Series(word_industrys)
df=pd.DataFrame(df.sort_values(ascending=False))

In [86]:
discard_list=[]
for word in vocabulary:
    if len(word)==1:discard_list.append(word)


In [87]:
vocabulary=vocabulary-(set(discard_list)-set(element))
len(vocabulary)

19404

In [88]:
# with open("D:3.AutoencoderForArticle/vocabulary.p","wb") as f:
#     pickle.dump(file=f,obj=vocabulary)

In [92]:
dict_vocabulary={v:i for i,v in enumerate(vocabulary)}

In [95]:
# with open("D:3.AutoencoderForArticle/dict_vocabulary","wb") as f:
#     pickle.dump(file=f,obj=dict_vocabulary)

#### Use vocabulary vectorize text

In [96]:
text=DataSet["title_token_plus_body_token"]
BOW_binary=np.zeros((len(text),len(vocabulary)),dtype="int8")

In [97]:
BOW_binary.shape

(154884, 19404)

In [98]:
for i,t in enumerate(tqdm_notebook(text)):
    for w in t.split():
        if w in dict_vocabulary:
            BOW_binary[i,dict_vocabulary[w]]=1

#### Saprsity

In [100]:
sparsity=np.sum(BOW_binary==1)/(BOW_binary.shape[0]*BOW_binary.shape[1])
print("sparsity:{:.2}%".format(round(sparsity,5)*100))

sparsity:0.53%


#### Save

In [101]:
# np.save("D:3.AutoencoderForArticle/BOW_binary_re_v02.npy",BOW_binary)

## Text to Vecotr multi-industry

<li> Load DataSet_vip_multi_industry
<li> Build Vocabulary(commenest 3700 words per industry)
<li> Use Vocanbulry to vectrize text  
<li> Sparcisity
<li> Save

In [110]:
with open("D:3.AutoencoderForArticle/DataSet_vip_multi_industry","rb") as f:
    DataSet=pickle.load(f)

In [111]:
with open("D:3.AutoencoderForArticle/dict_vocabulary","rb") as f:
    dict_vocabulary=pickle.load(f)

In [26]:
DataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127707 entries, 0 to 127706
Data columns (total 3 columns):
guid                           127707 non-null object
title_token_plus_body_token    127707 non-null object
indusrty_tags                  127707 non-null object
dtypes: object(3)
memory usage: 2.9+ MB


#### build BOW_binary_multi_v01

In [113]:
text=DataSet["title_token_plus_body_token"]
BOW_binary=np.zeros((len(text),len(vocabulary)),dtype="int8")

In [114]:
BOW_binary.shape

(127707, 19404)

In [115]:
for i,t in enumerate(tqdm_notebook(text)):
    for w in t.split():
        if w in dict_vocabulary:
            BOW_binary[i,dict_vocabulary[w]]=1

#### Saprsity

In [116]:
sparsity=np.sum(BOW_binary==1)/(BOW_binary.shape[0]*BOW_binary.shape[1])
print("sparsity:{:.2}%".format(round(sparsity,5)*100))

sparsity:0.61%


#### Save

In [118]:
# np.save("D:3.AutoencoderForArticle/BOW_binary_multi_v01.npy",BOW_binary)

In [117]:
BOW_binary.shape

(127707, 19404)